In [1]:
import random
import operator
import pandas as pd
from collections import Counter
import nltk
from joblib import Parallel, delayed
import tqdm
import string
import numpy as np
from joblib import Parallel, delayed
import re
from sklearn.metrics.pairwise import cosine_similarity as cosine

In [2]:
path_to_data = "/Users/zacharie/Documents/AICCours/MVA/textGraphData/"

In [3]:
training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0).values
training_info = pd.read_csv(path_to_data + 'training_info.csv', sep=',', header=0, parse_dates = [1]).values
test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0).values
test_info = pd.read_csv(path_to_data + 'test_info.csv', sep=',', header=0, parse_dates = [1]).values

In [7]:
integrate_training = []
for each_line in training:
    sender = each_line[0]
    ids = each_line[-1].split(' ')
    for my_id in ids:
        tmp = training_info[training_info[:,0]==int(my_id)]
        integrate_training.append([sender,tmp[0,0],tmp[0,1],tmp[0,2], tmp[0,3]])

In [8]:
integrate_training = pd.DataFrame(integrate_training, columns=['sender','mid','date','body','recipients'])

In [9]:
print integrate_training.index

RangeIndex(start=0, stop=43613, step=1)


In [10]:
integrate_test = []
for each_line in test:
    sender = each_line[0]
    ids = each_line[-1].split(' ')
    for my_id in ids:
        tmp = test_info[test_info[:,0]==int(my_id)]
        integrate_test.append([sender,tmp[0,0],tmp[0,1],tmp[0,2]])

In [11]:
integrate_test = pd.DataFrame(integrate_test, columns=['sender','mid','date','body'])

In [12]:
print integrate_test.index

RangeIndex(start=0, stop=2362, step=1)


In [14]:
# remove dashes and apostrophes from punctuation marks
punct = string.punctuation.replace('-', '').replace("'",'')
stpwds = nltk.corpus.stopwords.words('english')
# regex to match intra-word dashes and intra-word apostrophes
my_regex = re.compile(r"(\b[-']\b)|[\W_]")
stemmering = nltk.PorterStemmer()
# performs basic pre-processing
# note: we do not lowercase for consistency with Google News embeddings
def clean_string(string, punct=punct, my_regex=my_regex):
    # remove formatting
    str = re.sub('\s+', ' ', string)
    # remove punctuation (preserving dashes)
    str = ''.join(l for l in str if l not in punct)
    # remove dashes that are not intra-word
    str = my_regex.sub(lambda x: (x.group(1) if x.group(1) else ' '), str)
    # strip extra white space
    str = re.sub(' +',' ',str)
    # strip leading and trailing white space
    str = str.strip()
    # tokenize (split based on whitespace)
    tokens = str.split(' ')
    # remove stopwords
    tokens = [token for token in tokens if token not in stpwds]
    # remove tokens less than 2 characters in size
    # tokens = [token for token in tokens if len(token)>=2]
    tokens = [stemmering.stem(token.lower()) for token in tokens if len(token)>=2]
    return tokens

def stemmering_sentences_messages(X):
    sentences_stem = Parallel(n_jobs=4)(delayed(clean_string)(sentence) for sentence in tqdm.tqdm(X, desc="clean sentences"))
    return sentences_stem


In [15]:
integrate_training['body'] = stemmering_sentences_messages(integrate_training['body'])

clean sentences: 100%|██████████| 43613/43613 [01:33<00:00, 464.62it/s]

clean sentences: 100%|█████████▉| 43538/43613 [01:33<00:00, 648.03it/s]

clean sentences: 100%|█████████▉| 43436/43613 [01:33<00:00, 594.24it/s]

clean sentences:  99%|█████████▉| 43363/43613 [01:33<00:00, 551.81it/s]

clean sentences:  99%|█████████▉| 43272/43613 [01:33<00:00, 848.69it/s]

...

clean sentences:   3%|▎         | 1170/43613 [00:00<00:34, 1217.17it/s]

clean sentences:   2%|▏         | 878/43613 [00:00<00:30, 1388.87it/s]

clean sentences:   2%|▏         | 732/43613 [00:00<00:20, 2071.81it/s]

clean sentences:   1%|          | 440/43613 [00:00<00:21, 2041.72it/s]

clean sentences:   0%|          | 0/43613 [00:00<?, ?it/s]

In [16]:
integrate_test['body'] = stemmering_sentences_messages(integrate_test['body'])

clean sentences: 100%|██████████| 2362/2362 [00:04<00:00, 507.20it/s]

clean sentences:  94%|█████████▎| 2212/2362 [00:04<00:00, 284.62it/s]

clean sentences:  91%|█████████ | 2154/2362 [00:04<00:00, 265.29it/s]

clean sentences:  89%|████████▊ | 2096/2362 [00:04<00:01, 229.58it/s]

clean sentences:  84%|████████▍ | 1980/2362 [00:03<00:01, 208.89it/s]

...

clean sentences:  27%|██▋       | 646/2362 [00:00<01:26, 19.89it/s]

clean sentences:  22%|██▏       | 530/2362 [00:00<02:09, 14.12it/s]

clean sentences:  20%|█▉        | 472/2362 [00:00<03:09,  9.96it/s]

clean sentences:   0%|          | 1/2362 [00:00<05:38,  6.98it/s]

clean sentences:   0%|          | 0/2362 [00:00<?, ?it/s]

In [0]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import FeatureHasher
from sklearn.pipeline import Pipeline


In [0]:
hasher = HashingVectorizer(n_features=2 ** 18, stop_words='english', norm=None, ngram_range=(1, 2))
tfidf_transformer = TfidfTransformer()
vectorizer = Pipeline([('hasher', hasher), ('tf_idf', TfidfTransformer())])


In [52]:
X = training_info['body'].tolist() + test_info['body'].tolist()

In [53]:
import multiprocessing
from gensim.models.word2vec import Word2Vec
# 4 in pc
cpu_count = multiprocessing.cpu_count()
# the dimensionality of the feature vectors.
vocab_dim = 100
# ignore all words with total frequency lower than this. 
n_exposures = 5

In [54]:
print('Training a Word2vec model...')
model = Word2Vec(size=vocab_dim,
                 min_count=n_exposures,
                 workers=cpu_count)
model.build_vocab(X)
model.train(X)
# # replace our empty word vectors with GNews ones, to fill
# path_to_wv = '/Users/zacharie/Documents/AICCours/MVA/textGraphData/'
# # we load only the Google word vectors corresponding to our vocabulary
# model.intersect_word2vec_format(path_to_wv + 'GoogleNews-vectors-negative300.bin.gz', binary=True)
print('Training a Word2vec model is done!')

Training a Word2vec model...


Training a Word2vec model is done!


In [64]:
from sklearn.decomposition import PCA
# to preserve mapping, create (1) an array with the vectors of the words in vocab, (2) a list containg the actual words
model_array = np.empty(shape=model.syn0.shape)
mapping = []
for i, elt in enumerate(model.vocab.items()):
    word = elt[0]
    mapping.append(word)
    model_array[i,] = model[word]
my_pca = PCA(n_components=50)
wv_2d_values = my_pca.fit_transform(model_array)
# finally, create dictionary with the words in vocab as keys and the 2-dimensional projections as values
wv_2d = {}
for i, word in enumerate(mapping):
    wv_2d[word] =  wv_2d_values[i,]

In [55]:
################################
# create some handy structures #
################################
# convert training set to dictionary
emails_ids_per_sender = {}
for index, series in training.iterrows():
    row = series.tolist()
    sender = row[0]
    ids = row[1:][0].split(' ')
    emails_ids_per_sender[sender] = ids
# save all unique sender names
all_senders = emails_ids_per_sender.keys()
# create address book with frequency information for each user
address_books_messages = {}
i = 0
for sender, ids in emails_ids_per_sender.iteritems():
    rec2mids = {}
    for my_id in ids:
        recipients = training_info[training_info['mid']==int(my_id)]['recipients'].tolist()
        recipients = recipients[0].split(' ')
        # keep only legitimate email addresses
        recipients = [rec for rec in recipients if '@' in rec]
        for rec in recipients:
            if rec in rec2mids:
                rec2mids[rec].append(my_id)
            else:
                rec2mids[rec] = [my_id]
    address_books_messages[sender] = rec2mids
    if i % 20 == 0:
        print "For the moment, we have already done", i, " lines."
    i += 1

For the moment, we have already done 0  lines.


For the moment, we have already done 20  lines.


For the moment, we have already done 40  lines.


For the moment, we have already done 60  lines.


For the moment, we have already done 80  lines.


For the moment, we have already done 100  lines.


For the moment, we have already done 120  lines.


In [56]:
# print address_books.values()[0]
# save all unique recipient names
all_recs = list(set([elt for subdict in address_books_messages.values() for elt in subdict.keys()]))
# save all unique user names
all_users = []
all_users.extend(all_senders)
all_users.extend(all_recs)
all_users = list(set(all_users))

In [57]:
for sender, subdict in address_books_messages.iteritems():
    if len(subdict) < 10:
        print "sender", sender, "has only", len(subdict), "recipients."

sender alex@pira.com has only 1 recipients.
sender wsmith@wordsmith.org has only 1 recipients.
sender schwabalerts.marketupdates@schwab.com has only 1 recipients.


In [58]:
print "the number of all users:", len(all_users)
print "the number of all unique recipients:", len(all_recs)
print "the size of dictionary:",len(model.vocab)

the number of all users: 9783
the number of all unique recipients: 9779
the size of dictionary: 45194


In [80]:
# random.sample(all_users, 9)
# rec_occ = address_books_messages['jason.williams@enron.com']
# rec_occ_occ = {}
# for k, v in rec_occ.iteritems():
#     rec_occ_occ[k] = len(v)
# sorted_rec_occ = sorted(rec_occ_occ.items(), key=operator.itemgetter(1), reverse = True)
# print sorted_rec_occ

In [65]:
k_constant = 10
alpha = 1.0/3.0

In [66]:
def get_k_most(k_constant, sender, id_predict, alpha):
    s_1 = test_info[test_info['mid'] == int(id_predict)]['body'].tolist()[0]
    s_1_vec = [wv_2d[word].reshape(1,-1) for word in s_1 if word in wv_2d]
    if len(s_1_vec) == 0:
        rec_occ = address_books_messages[sender]
        rec_occ_tmp = {}
        for k, v in rec_occ.iteritems():
            rec_occ_tmp[k] = len(v)
        sorted_rec_occ = sorted(rec_occ_tmp.items(), key=operator.itemgetter(1), reverse = True)
        if len(sorted_rec_occ) < k_constant:
            return [elt[0] for elt in sorted_rec_occ] + random.sample(all_users, k_constant-len(sorted_rec_occ))
        else:
            return [elt[0] for elt in sorted_rec_occ[:k_constant]]
    else:
        p_1_embeddings = np.concatenate(s_1_vec)
        centroid_1 = np.mean(p_1_embeddings, axis=0).reshape(1,-1)
        k_most = []
        rec_list = []
        simil_freq_list = []
        for k_rec, v_mids_list in address_books_messages[sender].iteritems():
            tmp = []
            for mid in v_mids_list:
                s_2 = training_info[training_info['mid'] == int(mid)]['body'].tolist()[0]
                s_2_vec = [wv_2d[word].reshape(1,-1) for word in s_2 if word in wv_2d]
                if len(s_2_vec) == 0:
                    tmp.append(-1)
                    continue
                else:
                    p_2_embeddings = np.concatenate(s_2_vec)
                    centroid_2 = np.mean(p_2_embeddings, axis=0).reshape(1,-1)
                    tmp.append(round(cosine(centroid_1, centroid_2),6))
            similarity = np.max(tmp)
            rec_list.append(k_rec)
            simil_freq_list.append([similarity, len(v_mids_list)])
        simil_freq_array = np.array(simil_freq_list)
        rec_array = np.array(rec_list)
        simil_freq_array = (simil_freq_array - np.min(simil_freq_array, axis=0))/(np.max(simil_freq_array, axis=0)-np.min(simil_freq_array, axis=0))
        socre = alpha * simil_freq_array[:,0] + (1-alpha)*simil_freq_array[:,-1]
        if rec_array.shape[0] < k_constant:
            return rec_array[np.argsort(socre)[::-1]].tolist() + random.sample(all_users, k_constant-rec_array.shape[0])
        else:
            return rec_array[np.argsort(socre)[::-1]][:k_constant].tolist()


In [67]:
def predict_each_test_line(row):
    sender = row[0]
    # get IDs of the emails for which recipient prediction is needed
    ids_predict = row[1].split(' ')
    ids_predict = [int(my_id) for my_id in ids_predict]
    random_preds = []
    freq_preds = []
    # select k most frequent recipients for the user
    # k_most = [elt[0] for elt in address_books[sender][:k]]
    for id_predict in ids_predict:
        # select k users at random
        # random_preds.append(random.sample(all_users, k))
        # for the frequency baseline, the predictions are always the same
        k_most = get_k_most(k_constant, sender, id_predict, alpha)
        freq_preds.append(k_most)
    # predictions_per_sender[sender] = [ids_predict,random_preds,freq_preds]
    return (sender, [ids_predict, freq_preds])

In [68]:
def speedup(X):
    predictions_per_sender = Parallel(n_jobs=4, verbose=1)(delayed(predict_each_test_line)(row) for row in tqdm.tqdm(X, desc="speed up test"))
    return predictions_per_sender

In [0]:
import time
start = time.ctime()
predictions_per_sender = speedup(test.values)
end = time.ctime()
print "Total time:", start - end


speed up test:   0%|          | 0/125 [00:00<?, ?it/s]


speed up test:   1%|          | 1/125 [00:00<00:24,  5.08it/s]


speed up test:   6%|▋         | 8/125 [00:01<00:21,  5.32it/s]


speed up test:   7%|▋         | 9/125 [00:05<02:27,  1.27s/it]


speed up test:   8%|▊         | 10/125 [00:40<22:11, 11.58s/it]

...


speed up test:  35%|███▌      | 44/125 [10:16<14:56, 11.07s/it]


speed up test:  36%|███▌      | 45/125 [10:28<15:12, 11.40s/it]


speed up test:  37%|███▋      | 46/125 [10:28<10:40,  8.11s/it]


speed up test:  38%|███▊      | 47/125 [10:39<11:27,  8.82s/it]


speed up test:  38%|███▊      | 48/125 [11:02<16:51, 13.14s/it]

In [0]:
predictions_per_sender = dict(predictions_per_sender)

In [0]:
#################################################
# write predictions in proper format for Kaggle #
#################################################
path_to_results = "/Users/zacharie/Documents/AICCours/MVA/textGraphData/"

with open(path_to_results + 'predictions_frequency.txt', 'wb') as my_file:
    my_file.write('mid,recipients' + '\n')
    for sender, preds in predictions_per_sender.iteritems():
        ids = preds[0]
        freq_preds = preds[1]
        for index, my_preds in enumerate(freq_preds):
            my_file.write(str(ids[index]) + ',' + ' '.join(my_preds) + '\n')

In [0]:
def mean_average_precision(valid_2d, attempt_2d):
    def aux(valid_list, attempt_list, at=10):
        at = int(at)
        total_correct_nb = len(valid_list)
        mark_list = [1 if entry in valid_list else 0 for entry in attempt_list]
        score = 0.0
        for index, value in enumerate(mark_list):
            score += mark[:index+1].count(1) / (index+1)
        return score / min(total_correct_nb, at)
    # make sure they have the same dimensions.
    assert (len(valid_2d) == len(attempt_2d))
    total_score = 0.0
    for valid_list, attempt_list in zip(valide_2d, attempt_2d):
        total_score += aux(valid_list, attempt_list)
    return total_score / len(valid_2d)


In [68]:
def get_k_most(k, address_books_messages, sender, id_predict, alpha):
    test_body = test_info[test_info['mid'] == int(id_predict)]['body'].tolist()[0]
    k_most = []
    count = 0
    rec_list = []
    simil_freq_list = []
    # rec2similfreq = {}
    for k_rec, v_mids_list in address_books_messages[sender].iteritems():
        similarity = np.min([model.wmdistance(test_body, training_info[training_info['mid'] == int(mid)]['body'].tolist()[0]) for mid in v_mids_list])
        # rec2similfreq[k_rec] = (similarity, len(v_mids_list))
        rec_list.append(k_rec)
        simil_freq_list.append([similarity, len(v_mids_list)])
    simil_freq_list = np.array(simil_freq_list)
    rec_list = np.array(rec_list)
    simil_freq_list[:,0] = (np.max(simil_freq_list[:,0]) - simil_freq_list[:,0]) / (np.max(simil_freq_list[:,0]) - np.min(simil_freq_list[:,0]))
    simil_freq_list[:,-1] = (simil_freq_list[:,-1] - np.min(simil_freq_list[:,-1])) / (np.max(simil_freq_list[:,-1])-np.min(simil_freq_list[:,-1]))
    socre = alpha * simil_freq_list[:,0] + (1-alpha)*simil_freq_list[:,-1]
    return simil_freq_list[np.argsort(socre)[::-1]][:k].tolist()


In [0]:
#############
# baselines #
#############
# will contain email ids, predictions for random baseline, and predictions for frequency baseline
predictions_per_sender = {}
# number of recipients to predict
k = 10
for row in test.values:
    sender = row[0]
    # get IDs of the emails for which recipient prediction is needed
    ids_predict = row[1].split(' ')
    ids_predict = [int(my_id) for my_id in ids_predict]
    random_preds = []
    freq_preds = []
    # select k most frequent recipients for the user
    # k_most = [elt[0] for elt in address_books[sender][:k]]
    for id_predict in ids_predict:
        # select k users at random
        random_preds.append(random.sample(all_users, k))
        # for the frequency baseline, the predictions are always the same
        k_most = get_k_most(k, address_books_messages, sender, id_predict, 1/2.0)
        freq_preds.append(k_most)
    predictions_per_sender[sender] = [ids_predict,random_preds,freq_preds]
